In [1]:
from collections import defaultdict
from itertools import product

def int_to_bin(i, n):
    return format(i, '0{}b'.format(n))

def bitcount(s):
    return s.count('1')

def combine(a, b):
    # a,b は 0/1/- の同長文字列。1ビットだけ違えば結合してダッシュ化
    diff = 0
    out = []
    for x, y in zip(a, b):
        if x == y:
            out.append(x)
        elif x != y and x != '-' and y != '-':
            diff += 1
            out.append('-')
        else:
            return None  # ダッシュ含みの不適合や2bit以上の差
        if diff > 1:
            return None
    return ''.join(out) if diff == 1 else None

def covers(term, mint):
    # term: 0/1/-, mint: 0/1（同長）
    return all(t == m or t == '-' for t, m in zip(term, mint))

def qm_minimize(num_vars, minterms, dont_cares=None):
    if dont_cares is None:
        dont_cares = []
    minterms = sorted(set(minterms))
    dcs = sorted(set(dont_cares))
    universe = set(minterms)

    # 初期グループ：1の個数で分ける
    groups = defaultdict(list)
    origin = {}  # 項 -> カバーする minterm 集合
    for i in minterms + dcs:
        b = int_to_bin(i, num_vars)
        groups[bitcount(b)].append(b)
        origin[b] = frozenset([i])

    # 結合ラウンド
    prime_implicants = set()
    current = groups
    while True:
        next_groups = defaultdict(list)
        used = set()
        keys = sorted(current.keys())
        for k in keys:
            if k+1 not in current: 
                continue
            for a in current[k]:
                for b in current[k+1]:
                    c = combine(a, b)
                    if c:
                        used.add(a); used.add(b)
                        if c not in next_groups[bitcount(c.replace('-', ''))]:
                            next_groups[bitcount(c.replace('-', ''))].append(c)
                        # 出自 minterm の統合
                        origin[c] = origin.get(c, frozenset()) | origin[a] | origin[b]
        # 使用されなかったものは PI 候補へ
        for k in keys:
            for a in current[k]:
                if a not in used:
                    prime_implicants.add(a)
        if not next_groups:
            break
        current = next_groups

    # PI -> minterm カバレッジ
    pis = list(prime_implicants)
    cover = {pi: set(x for x in origin[pi] if x in universe) for pi in pis}
    # 1つも minterm を覆わない PI は除外
    pis = [pi for pi in pis if cover[pi]]

    # 本質素因子を先に選ぶ
    remaining = set(universe)
    essential = set()
    while True:
        chosen = None
        for m in list(remaining):
            holders = [pi for pi in pis if m in cover[pi]]
            if len(holders) == 1:
                chosen = holders[0]
                essential.add(chosen)
                remaining -= cover[chosen]
        if not chosen:
            break

    if not remaining:
        result = sorted(essential)
        return result

    # ペトリック法：未カバー列の“和の積”を構成
    sums = []
    for m in sorted(remaining):
        alt = [pi for pi in pis if m in cover[pi]]
        sums.append(set((pi,) for pi in alt))  # 各要素は PI のタプル

    # 逐次で直積しつつ剪定（項数→リテラル数）
    def score(combo):
        # 項数（PI数）を第一指標、総リテラル数を第二指標
        def lit_count(term):
            return sum(1 for ch in term if ch != '-')
        s1 = len(combo)
        s2 = sum(lit_count(pi) for pi in combo)
        return (s1, s2)

    prod_set = sums[0]
    for s in sums[1:]:
        new = set()
        for a, b in product(prod_set, s):
            merged = tuple(sorted(set(a) | set(b)))
            new.add(merged)
        # 吸収・剪定：スコア優越で劣後を落とす
        new = list(new)
        new.sort(key=score)
        kept = []
        seen = set()
        for c in new:
            sc = score(c)
            if any(set(c) >= set(d) for d in kept):
                continue
            kept = [d for d in kept if not set(d) >= set(c)]
            kept.append(c)
        prod_set = set(kept)

    best = min(prod_set, key=score)
    result = sorted(set(essential) | set(best))
    return result

def term_to_expr(term, vars_):
    out = []
    for v, t in zip(vars_, term):
        if t == '1':
            out.append(v)
        elif t == '0':
            out.append(f"\\overline{{{v}}}")
    return ''.join(out) if out else '1'

# 使い方例
if __name__ == "__main__":
    # 例：F(A,B,C,D)=Σm(1,3,7,9,11,15), d=Σm(0,2,8,10)
    num_vars = 4
    mins = [3,5,6,7,13,14,15]
    dcs  = [4,10,12]
    pis = qm_minimize(num_vars, mins, dcs)
    vars_ = ['A','B','C','D']
    print("Minimal SOP = " + " + ".join(term_to_expr(t, vars_) for t in pis))


Minimal SOP = B + \overline{A}CD
